# Assignment 4 - Syntax

This assigment has two parts. The first part is a continuation of assigment 2 about text classification. You will develop a text classifier based on syntactic features, namely POS and dependency, and test it on the 20 newsgroup dataset. The second part will focus on context free grammar and constituency parsing.  


## Part 1 - Text Classification with Syntactic Features

First, let's install Spacy and download the English pipeline.

In [1]:
import spacy

!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 22.0 MB/s  0:00:18:00:0100:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


We are now loading the 20 newsgroup dataset, as we did in assignment 2. Again, we will only keep two topics, 'rec.sport.baseball' and 'rec.sport.hockey'.

In [2]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(
    subset='train', remove=('headers', 'footers', 'quotes'),
    categories=['rec.sport.baseball','rec.sport.hockey'])

newsgroups_test = fetch_20newsgroups(
    subset='test', remove=('headers', 'footers', 'quotes'),
    categories=['rec.sport.baseball','rec.sport.hockey'])

# Remove empty documents
for s, name in zip([newsgroups_train, newsgroups_test], ["train", "test"]):
  empty_indices = {i for i, doc in enumerate(s.data) if len(doc) == 0}
  orig_len = len(s.data)
  for k in ['data', 'filenames', 'target', 'DESCR']:
    s[k] = [s[k][i] for i in range(orig_len) if i not in empty_indices]
  print(f"Removed {len(empty_indices)} empty documents from the {name} set. Before: {orig_len}. After: {len(s.data)}.")

Removed 26 empty documents from the train set. Before: 1197. After: 1171.
Removed 19 empty documents from the test set. Before: 796. After: 777.


Here we explore how to vectorize documents with two syntatic features: part-of-speech tags and syntactic dependencies, in addition to the bag-of-words features (same as assignment 2: lower-cased lemmas, no punctuation or stop words).

We will need to build the pipeline for the syntax-based classifier. Please complete the code below to implement the syntax-based document vectorizer __SyntaxVectorizer__. This vectorizer will extract POS and dependency features and represent each of them as a bag-of-words.

In [3]:
import string
import numpy as np

from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer


class SyntaxVectorizer(BaseEstimator):
    def __init__(self, nlp):
        self.nlp = nlp
        self.pos_vectorizer = CountVectorizer()
        self.dep_vectorizer = CountVectorizer()
        self.word_vectorizer = CountVectorizer()

    def get_features(self, X):
        ####################################
        #   Your code here
        ####################################

        ####################################

        pos_features = []
        dep_features = []
        word_features = []

        # process documents in batch for speed
        for doc in self.nlp.pipe(X):
            # POS and dependency sequences (exclude punctuation/space)
            pos_tokens = [token.pos_ for token in doc if not (token.is_punct or token.is_space)]
            dep_tokens = [token.dep_ for token in doc if not (token.is_punct or token.is_space)]
            pos_features.append(" ".join(pos_tokens) if pos_tokens else "")
            dep_features.append(" ".join(dep_tokens) if dep_tokens else "")

            # Word features: lower-cased lemmas, no punctuation or stop words
            words = []
            for token in doc:
                if token.is_punct or token.is_space:
                    continue
                lemma = token.lemma_.lower()
                if lemma in STOP_WORDS or lemma in string.punctuation:
                    continue
                words.append(lemma)
            word_features.append(" ".join(words) if words else "")

        return pos_features, dep_features, word_features

    def transform(self, X):
        # Gets a list of documents, each document represented as string
        # concatenation of all POS (dep) tags, and transforms them into
        # count vectors (bag-of-features).
        pos_features, dep_features, word_features = self.get_features(X)
        X_pos = self.pos_vectorizer.transform(pos_features).toarray()
        X_dep = self.dep_vectorizer.transform(dep_features).toarray()
        X_word = self.word_vectorizer.transform(word_features).toarray()
        X_features = np.concatenate((X_pos, X_dep, X_word), axis=-1)
        return X_features

    def fit(self, X, y=None):
        pos_features, dep_features, word_features = self.get_features(X)
        self.pos_vectorizer.fit(pos_features)
        self.dep_vectorizer.fit(dep_features)
        self.word_vectorizer.fit(word_features)
        return self


# Load the English pipeline en_core_web_lg
nlp = spacy.load("en_core_web_lg")

# Create pipeline for the syntax-based classifier
classifier = LogisticRegression(max_iter=1000)
pipe = Pipeline([
    ("vectorizer", SyntaxVectorizer(nlp)), ("classifier", classifier)])

Now let's train this classifier.

In [4]:
pipe.fit(newsgroups_train.data, newsgroups_train.target)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,nlp,<spacy.lang.e...t 0x1460a8980>
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1


Next, let's predict and evaluate the test set. How does the performance compare to our BOW and CBOW classifiers from assignment 2?

In [5]:
from sklearn import metrics

preds = pipe.predict(newsgroups_test.data)
print(f"Logistic Regression Accuracy: {metrics.accuracy_score(newsgroups_test.target, preds)*100:.2f}%")

Logistic Regression Accuracy: 86.87%


Finally, let's print the most useful features. Check which type of features were useful - did syntactic features help? You may change k to print the most useful k features.

In [6]:
k = 10

pos_features = sorted(pipe["vectorizer"].pos_vectorizer.vocabulary_.keys())
dep_features = sorted(pipe["vectorizer"].dep_vectorizer.vocabulary_.keys())
word_features = sorted(pipe["vectorizer"].word_vectorizer.vocabulary_.keys())
all_features = [f"pos:{f}" for f in pos_features] + [f"dep:{f}" for f in dep_features] + [f"w:{f}" for f in word_features]

coeff = pipe.steps[1][1].coef_
idx = (-coeff).argsort()[0]
desc_feature = [all_features[i] for i in idx]
top_features = desc_feature[:k]
print(top_features)

['w:hockey', 'w:playoff', 'w:nhl', 'w:goal', 'w:mask', 'w:detroit', 'w:cup', 'w:leafs', 'w:puck', 'w:ice']


## Part 2 - Context Free Grammar

In this part, you will be working with constituency parsing with structures defined by context-free grammars.

Specifically, you will first convert a set of production rules into Chomsky Normal Form, before applying the Cocke–Kasami-Younger (CKY) Algorithm to obatin all possible parses for a given input sentence.

### Chomsky Normal Form

The following function takes a list of production rules and converts them into Chomsky Normal Forms (CNF). As a reminder, in CNF, a non-terminal can generate:

1. A terminal (e.g.; X -> x); or
2. Two non-terminals (e.g.; X->YZ).

Complete `convert_rule` to generate new rules for the following cases:

1. A non-terminal generates more than two non-terminals, e.g. `A -> B C D`. In that case, split it to `A -> A0 D` and `A0 -> B C`, recursively checking `A -> A0 D`. Don't forget to add the new non-terminal to `non_terminals`.
2. A non-terminal generates more a mix of terminals and non-terminals, e.g. `A -> B b`. In that case, split it to `A -> B A1`, and `A1 -> b`, recursively checking `A -> B A1`. Don't forget to add the new non-terminal to `non_terminals`.

In [7]:
from collections import defaultdict


def convert_rule(rule, non_terminals):
  """
  Gets a rule (as a tuple) and returns a list of rules to replace it
  """
  # A terminal generates a non-terminal (e.g. A -> a), return it
  if len(rule) == 2 and rule[1][0] == "'":
    return [tuple(rule)]

  # A terminal generates two non-terminals (e.g. A -> B C), return it.
  # We also return unary rules (e.g. A -> B) which will be taken care of later.
  elif len(rule) <= 3 and all([x[0] != "'" for x in rule]):
    return [tuple(rule)]

  ####################################
  #   Your code here
  ####################################

  ####################################
  lhs = rule[0]
  rhs = list(rule[1:])  # RHS symbols

  new_rules = []

  # helper to generate a fresh non-terminal for a given lhs
  def fresh_nt(base):
    # count existing numbered variants for base (e.g., A0, A1, ...)
    cnt = sum(1 for n in non_terminals if n.startswith(base) and n[len(base):].isdigit())
    name = f"{base}{cnt}"
    non_terminals.append(name)
    return name

  # Replace any terminal symbols in RHS (when mixed with non-terminals)
  # by introducing a fresh non-terminal that produces that terminal.
  for i, sym in enumerate(rhs):
    if sym[0] == "'" and len(rhs) > 1:
      nt = fresh_nt(lhs)
      new_rules.append((nt, sym))
      rhs[i] = nt

  # If RHS has more than 2 symbols, iteratively factor the first two into a new NT
  # until RHS length is <= 2. This follows the pattern:
  # A -> X1 X2 X3 X4  => create A0 -> X1 X2, replace RHS with [A0, X3, X4], ...
  while len(rhs) > 2:
    nt = fresh_nt(lhs)
    # create rule nt -> first second
    new_rules.append((nt, rhs[0], rhs[1]))
    # replace first two symbols with the new non-terminal
    rhs = [nt] + rhs[2:]

  # finally add the transformed rule for the original LHS
  new_rules.append(tuple([lhs] + rhs))

  return new_rules


def convert_rules(grammar_rules):
    """
    Converts a list context-free grammar rules into the
    formatted Chomsky Normal Form.
    """
    grammar_rules = [rule.replace("->", "").split() for rule in grammar_rules]
    non_terminals = list({rule[0] for rule in grammar_rules})
    new_rules = [convert_rule(rule, non_terminals) for rule in grammar_rules]
    new_rules = [rule for rule_lst in new_rules for rule in rule_lst]

    # Recursively combine the unary rules (e.g. A -> B)
    # with an existing rule (if possible). E.g. if B -> b and B -> C D,
    # we will add A -> b and A -> C D to the rules.
    unary_rules = {rule for rule in new_rules
                   if len(rule) == 2 and rule[1][0] != "'"}

    # Convert to a dictionary from the LHS to the rest of the rule,
    # for better processing of unary rules
    rules_dict = defaultdict(list)
    [rules_dict[rule[0]].append(list(rule[1:])) for rule in new_rules if rule not in unary_rules]

    for unary_rule in unary_rules:
      lhs, rhs = unary_rule
      rules_dict[lhs].extend(rules_dict[rhs])

    formatted_rules = [tuple([lhs] + rhs)
                       for lhs, rhs_lst in rules_dict.items()
                       for rhs in rhs_lst]

    return formatted_rules

You can test your implementation with the following code:

In [8]:
grammar_rules = [
    "A -> B",
    "A -> B C",
    "A -> 'a'",
    "A -> B C D",
    "A -> C D E",
    "B -> C D E",
    "C -> D",
    "D -> 'b'",
]

assert set(convert_rules(grammar_rules)) == set([
    ('A', 'B', 'C'),
    ('A', "'a'"),
    ('A', 'A0', 'D'), # from A -> B C D
    ('A0', 'B', 'C'), # from A -> B C D
    ('A', 'A1', 'E'), # from A -> C D E
    ('A1', 'C', 'D'), # from A -> C D E
    ('B', 'B0', 'E'), # from B -> C D E
    ('B0', 'C', 'D'), # from B -> C D E
    ('D', "'b'"),
    ('C', "'b'"), # from C -> D and D -> b
    ('A', 'B0', 'E'), # from A -> B and B -> C D E
])

### CKY Parsing

The following function outputs all possible parses based on the input text and a list of CNF production rules.

Each entry in the cell is represented by the `Node` object, which stores the left and right child of the non-terminal (only left child when generated from terminals). This is to keep track of the entry from which it was derived, and is used to recursively retrieve its component constituents.

Using CKY, each cell in the parse triangle can be filled in $\mathcal{O}(n)$ time, where the number of spans to consider is defined by the variable `n_spans`, where higher levels considers more spans. Please complete in the missing code where indicated by assigning the left and right cell according to the appropriate indices in the parse triangle, so the existing cell considers all places where the input might be split in two.

In [16]:
class Node(object):
    """
    Data structure used for storing information about a non-terminal symbol.
    """
    def __init__(self, symbol, child1, child2=None):
        self.symbol = symbol
        self.child1 = child1
        self.child2 = child2

    def __repr__(self):
        return self.symbol


def CKY_parse(text, rules):
    """
    Performs Constituency Parsing with the CKY algorithm.

    Args:
        text (str): Input sentence, where terminals are separated by white spaces
        rules (List[List[str]]): List of production rules in CNF (see `simple_CNF` for format)

    Returns:
        parse_triangle (List[List[List[Node]]]): Data structure for CKY parsing,
        the first row `parse_triangle[0]` represents the non-terminals for
        generating the terminals in the input text, where each cell
        `parse_triangle[i][j]` is represented by a list containing all possible
        symbols based on the previous cells.
    """
    tokens = text.split()
    length = len(tokens)

    # Data structure for storing the subtrees
    parse_triangle = [[[] for x in range(length - i)] for i in range(length)]

    for i, tok in enumerate(tokens):

        # Find out which non-terminals can generate the terminals
        # in the input text and put them into the parse table.
        # One terminal could be generated by multiple non-terminals,
        # therefore the parse table will contain a list of non terminals.
        for rule in rules:
            if f"'{tok}'" == rule[1]:
                parse_triangle[0][i].append(Node(rule[0], tok))

    # For each span length from 2 to the entire string
    for row_idx in range(1, length):

        # For each start index
        for cell_idx in range(length - row_idx):

            # For each partition index
            for span_idx in range(row_idx):

                # Index the parse_triangle for the left and right cell to consider
                # (hint: the indices is based on `row_idx`, `cell_idx`, and `span_idx`)

                ####################################
                #   Your code here
                ####################################
                # left span length = span_idx + 1 → left row = span_idx
                left_cell = parse_triangle[span_idx][cell_idx]
                # right span length = (row_idx+1) - (span_idx+1) = row_idx - span_idx
                # right row index = (right span length) - 1 = row_idx - span_idx - 1
                right_cell = parse_triangle[row_idx - span_idx - 1][cell_idx + span_idx + 1]
                ####################################

                # For any rule where a non-terminal can be produced from the symbols
                # in the left and right cell
                for rule in rules:
                    if len(rule) == 3:

                        # Update the current cell based if a non-terminal can be produced.

                        ####################################
                        #   Your code here
                        ####################################
                        left_nodes = [n for n in left_cell if n.symbol == rule[1]]
                        right_nodes = [n for n in right_cell if n.symbol == rule[2]]
                        ####################################
                        parse_triangle[row_idx][cell_idx].extend(
                            [Node(rule[0], left, right) for left in left_nodes for right in right_nodes]
                        )

    return parse_triangle

Now we can test our CKY implementation. The function `print_tree` prints out all possible parse trees using recursion given the `parse_triangle` data structure returned by the `CKY_parse` function.

In [17]:
def generate_tree(node):
    """
    Generates the string representation of the parse tree.
    :param node: the root node.
    :return: the parse tree in string form.
    """
    if node.child2 is None:
        return f"[{node.symbol} '{node.child1}']"
    return f"[{node.symbol} {generate_tree(node.child1)} {generate_tree(node.child2)}]"

def print_tree(parsed_triangle, rules, output=True):
    """
    Print the parse tree starting with the start symbol. Alternatively it returns the string
    representation of the tree(s) instead of printing it.
    """
    start_symbol = rules[0][0]
    final_nodes = [n for n in parsed_triangle[-1][0] if n.symbol == start_symbol]
    print("Possible parse(s):")
    trees = [generate_tree(node) for node in final_nodes]
    if output:
        for tree in trees:
            print(tree)

Let's combine everything we implemented in this part to test our constituency parsing algorithm. Given the list of CFG production rules `grammar_rules` and a text input `text_input`, print out all possible parse trees using the functions defined in the previous questions.

In [ ]:
grammar_rules = [
    "S -> NP VP",
    "PP -> P NP",
    "NP -> Det N",
    "NP -> Det N PP",
    "VP -> V NP",
    "VP -> VP PP",
    "NP -> 'I'",
    "Det -> 'an'",
    "Det -> 'my'",
    "N -> 'elephant'",
    "N -> 'pajamas'",
    "V -> 'shot'",
    "P -> 'in'",
]

text_input = "I shot an elephant in my pajamas"

####################################
#   Your code here
####################################
# Convert grammar to CNF, run CKY, and print parses
cnf_rules = convert_rules(grammar_rules)
# ensure start symbol 'S' appears first (print_tree expects rules[0][0] as start)
cnf_rules = sorted(cnf_rules, key=lambda r: (0 if r[0] == 'S' else 1, r))
print("CNF rules:")
for r in cnf_rules:
    print(r)
# Run CKY
parsed = CKY_parse(text_input, cnf_rules)

# Print parse trees
print_tree(parsed, cnf_rules)

####################################

CNF rules:
('S', 'NP', 'VP')
('Det', "'an'")
('Det', "'my'")
('N', "'elephant'")
('N', "'pajamas'")
('NP', "'I'")
('NP', 'Det', 'N')
('NP', 'NP0', 'PP')
('NP0', 'Det', 'N')
('P', "'in'")
('PP', 'P', 'NP')
('V', "'shot'")
('VP', 'V', 'NP')
('VP', 'VP', 'PP')
Possible parse(s):
[S [NP 'I'] [VP [V 'shot'] [NP [NP0 [Det 'an'] [N 'elephant']] [PP [P 'in'] [NP [Det 'my'] [N 'pajamas']]]]]]
[S [NP 'I'] [VP [VP [V 'shot'] [NP [Det 'an'] [N 'elephant']]] [PP [P 'in'] [NP [Det 'my'] [N 'pajamas']]]]]
